# Worksheet 08

Name:  Yifei Zhou  
UID: u25549843 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.239014420387377, 4.138691208009267, 6.357646317073613, 5.3304250370681565, 4.428695932535211, 6.057103782665105, 2.748502542823276, 3.793195791456715, 5.003250442593126, 5.157532784999163]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[5.773410649343614, 8.357190078077839, 7.867754523139237, 9.25381077546088, 6.6317674222249865, 7.75546902337564, 6.3182184089382645, 7.035623439783415, 7.050576014563395, 8.523764873205312]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.157532784999163, 5.003250442593126, 8.523764873205312, 3.793195791456715, 7.050576014563395, 2.748502542823276, 7.035623439783415, 6.057103782665105, 6.3182184089382645, 4.428695932535211]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [12]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) ,len(s2) / (len(s1) + len(s2))  ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.157532784999163, 5.003250442593126, 3.793195791456715, 2.748502542823276, 4.428695932535211]
[8.523764873205312, 7.050576014563395, 7.035623439783415, 6.057103782665105, 6.3182184089382645]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.226235498881498,  mean_2 = 6.997057303831099
var_1 = 0.7766550325407351,  var_2 = 8.413357968543549


d:\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [14]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]* prob_s[0]/prob_x )
    prob_s1_x.append( pdf_i[1]* prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.157532784999163
probability of observing that point if it came from cluster 0 =  0.2502948284647668
probability of observing that point if it came from cluster 1 =  0.046297761767311
point =  5.003250442593126
probability of observing that point if it came from cluster 0 =  0.31141057777857
probability of observing that point if it came from cluster 1 =  0.04610475300702452
point =  8.523764873205312
probability of observing that point if it came from cluster 0 =  1.1534473001690347e-07
probability of observing that point if it came from cluster 1 =  0.04664341591664174
point =  3.793195791456715
probability of observing that point if it came from cluster 0 =  0.43971794116825
probability of observing that point if it came from cluster 1 =  0.04410130034990973
point =  7.050576014563395
probability of observing that point if it came from cluster 0 =  0.0006902842084842794
probability of observing that point if it came from cluster 1 =  0.047416763972833564
point =  2.7485025

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [16]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.546645872844319,  mean_2 = 6.621066561206644
var_1 = 0.8793178402837301,  var_2 = 6.7404206748700615


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [17]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]* prob_s[0]/prob_x )
    prob_s1_x.append( pdf_i[1]* prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  5.157532784999163
probability of observing that point if it came from cluster 0 =  0.35641719977172587
probability of observing that point if it came from cluster 1 =  0.05780770857243943
point =  5.003250442593126
probability of observing that point if it came from cluster 0 =  0.3964715268813267
probability of observing that point if it came from cluster 1 =  0.05750605851370244
point =  8.523764873205312
probability of observing that point if it came from cluster 0 =  1.6388820906644147e-05
probability of observing that point if it came from cluster 1 =  0.05687483016440633
point =  3.793195791456715
probability of observing that point if it came from cluster 0 =  0.31429222832205445
probability of observing that point if it came from cluster 1 =  0.054200375337764124
point =  7.050576014563395
probability of observing that point if it came from cluster 0 =  0.007870034116903755
probability of observing that point if it came from cluster 1 =  0.05906651583014785
point =  2.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [18]:
probabilities= zip(data, prob_s0_x, prob_s1_x)
for prob in probabilities:
    if prob[1]>prob[2]:
        print(str(prob[0])+" is in cluster 0")
    else:
        print(str(prob[0])+" is in cluster 1")

5.157532784999163 is in cluster 0
5.003250442593126 is in cluster 0
8.523764873205312 is in cluster 1
3.793195791456715 is in cluster 0
7.050576014563395 is in cluster 1
2.748502542823276 is in cluster 0
7.035623439783415 is in cluster 1
6.057103782665105 is in cluster 0
6.3182184089382645 is in cluster 0
4.428695932535211 is in cluster 0
